Module imports

In [1]:
import geopandas as gpd
import fiona

# HEX 5

Hex imports

In [2]:
%%time
# Define the path to the geodatabase
inputs_gdb = r'C://Research/Grid_effort/H3Grid_Inputs.gdb'
hex_gdb = r"C://Research/Grid_effort/H3_5_10_Grd.gdb"
hexLevel = '5'

# Read in the H3_5 polygon grid layer as the base layer
base_layer = gpd.read_file(hex_gdb, layer='H3_'+hexLevel)
base_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)

CPU times: total: 672 ms
Wall time: 679 ms


Hex to counties

In [3]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='tj_2021_us_st_cnt')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

CPU times: total: 1.53 s
Wall time: 1.54 s


In [4]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

CPU times: total: 422 ms
Wall time: 423 ms


In [5]:
%%time
output_layer.to_file('h'+hexLevel+'_tj2021.gdb',driver='OpenFileGDB')

CPU times: total: 828 ms
Wall time: 831 ms


In [6]:
del joined_layer, output_layer

Hex to estuarine

In [7]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='Estuarine_Drainage_Areas')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

CPU times: total: 266 ms
Wall time: 259 ms


In [8]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

CPU times: total: 234 ms
Wall time: 240 ms


In [9]:
%%time
output_layer.to_file('h'+hexLevel+'_Estuarine.gdb',driver='OpenFileGDB')

CPU times: total: 562 ms
Wall time: 584 ms


In [10]:
del joined_layer, output_layer

Hex to huc8

In [11]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='WBDHU8')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

CPU times: total: 24.1 s
Wall time: 24.2 s


In [12]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

CPU times: total: 1.19 s
Wall time: 1.18 s


In [13]:
%%time
output_layer.to_file('h'+hexLevel+'_WBDHU8.gdb',driver='OpenFileGDB')

CPU times: total: 703 ms
Wall time: 705 ms


In [14]:
del joined_layer, output_layer

Hex to census

In [15]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='dtl_cnty_Census_ESRI')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

CPU times: total: 8.83 s
Wall time: 8.83 s


In [16]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

CPU times: total: 5.39 s
Wall time: 5.39 s


In [17]:
%%time
output_layer.to_file('h'+hexLevel+'_Census.gdb',driver='OpenFileGDB')

CPU times: total: 688 ms
Wall time: 690 ms


In [18]:
del joined_layer, output_layer

Hex to huc12

In [19]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='WBDHU12')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

CPU times: total: 8.66 s
Wall time: 8.68 s


In [20]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

CPU times: total: 703 ms
Wall time: 665 ms


In [21]:
%%time
output_layer.to_file('h'+hexLevel+'_WBDHU12.gdb',driver='OpenFileGDB')

CPU times: total: 1.59 s
Wall time: 1.62 s


In [22]:
del joined_layer, output_layer

Hex to all at once

In [23]:
%%time
layers_to_join = ['tj_2021_us_st_cnt', 'Estuarine_Drainage_Areas', 'WBDHU8', 'dtl_cnty_Census_ESRI', 'WBDHU12']
layer_gdfs = {layer: gpd.read_file(inputs_gdb, layer=layer) for layer in layers_to_join}

CPU times: total: 43.7 s
Wall time: 43.8 s


In [24]:
%%time
for name, gdf in layer_gdfs.items():
    # Ensure the CRS is the same between the base layer and the current layer
    if not gdf.crs == base_layer.crs:
        gdf = gdf.to_crs(base_layer.crs)
    if 'Shape_Length' in gdf.columns:
        gdf.drop('Shape_Length', axis=1, inplace=True)
    if 'Shape_Area' in gdf.columns:
        gdf.drop('Shape_Area', axis=1, inplace=True)

    # Perform spatial join (inner join by default)
    base_layer = gpd.sjoin(base_layer, gdf, how="left")
    
    # Drop the `index_right` column if it exists
    if 'index_right' in base_layer.columns:
        base_layer.drop('index_right', axis=1, inplace=True)

CPU times: total: 23 s
Wall time: 23 s


In [25]:
%%time
base_layer.to_file('h'+hexLevel+'_allAtOnce.gdb',driver='OpenFileGDB')

CPU times: total: 23.4 s
Wall time: 23.4 s


In [26]:
del layers_to_join, layer_gdfs, gdf, base_layer

# HEX 6

Hex imports

In [27]:
%%time
# Define the path to the geodatabase
inputs_gdb = r'C://Research/Grid_effort/H3Grid_Inputs.gdb'
hex_gdb = r"C://Research/Grid_effort/H3_5_10_Grd.gdb"
hexLevel = '6'

# Read in the H3_5 polygon grid layer as the base layer
base_layer = gpd.read_file(hex_gdb, layer='H3_'+hexLevel)
base_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)

CPU times: total: 4.25 s
Wall time: 4.24 s


Hex to counties

In [28]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='tj_2021_us_st_cnt')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

CPU times: total: 1.56 s
Wall time: 1.55 s


In [29]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

CPU times: total: 2.05 s
Wall time: 2.05 s


In [30]:
%%time
output_layer.to_file('h'+hexLevel+'_tj2021.gdb',driver='OpenFileGDB')

CPU times: total: 4.12 s
Wall time: 4.13 s


In [31]:
del joined_layer, output_layer

Hex to estuarine

In [32]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='Estuarine_Drainage_Areas')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

CPU times: total: 234 ms
Wall time: 226 ms


In [33]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

CPU times: total: 1.3 s
Wall time: 1.3 s


In [34]:
%%time
output_layer.to_file('h'+hexLevel+'_Estuarine.gdb',driver='OpenFileGDB')

CPU times: total: 3.38 s
Wall time: 3.38 s


In [35]:
del joined_layer, output_layer

Hex to huc8

In [36]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='WBDHU8')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

CPU times: total: 25.6 s
Wall time: 25.6 s


In [37]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

CPU times: total: 6.5 s
Wall time: 6.51 s


In [38]:
%%time
output_layer.to_file('h'+hexLevel+'_WBDHU8.gdb',driver='OpenFileGDB')

CPU times: total: 3.7 s
Wall time: 3.69 s


In [39]:
del joined_layer, output_layer

Hex to census

In [40]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='dtl_cnty_Census_ESRI')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

CPU times: total: 9.2 s
Wall time: 9.21 s


In [41]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

CPU times: total: 35.7 s
Wall time: 35.7 s


In [42]:
%%time
output_layer.to_file('h'+hexLevel+'_Census.gdb',driver='OpenFileGDB')

CPU times: total: 3.8 s
Wall time: 3.9 s


In [43]:
del joined_layer, output_layer

Hex to huc12

In [44]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='WBDHU12')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

CPU times: total: 8.86 s
Wall time: 8.89 s


In [45]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

CPU times: total: 1.92 s
Wall time: 1.92 s


In [46]:
%%time
output_layer.to_file('h'+hexLevel+'_WBDHU12.gdb',driver='OpenFileGDB')

CPU times: total: 5.5 s
Wall time: 5.52 s


In [47]:
del joined_layer, output_layer

Hex to all at once

In [48]:
%%time
layers_to_join = ['tj_2021_us_st_cnt', 'Estuarine_Drainage_Areas', 'WBDHU8', 'dtl_cnty_Census_ESRI', 'WBDHU12']
layer_gdfs = {layer: gpd.read_file(inputs_gdb, layer=layer) for layer in layers_to_join}

CPU times: total: 46.2 s
Wall time: 46.3 s


In [49]:
%%time
for name, gdf in layer_gdfs.items():
    # Ensure the CRS is the same between the base layer and the current layer
    if not gdf.crs == base_layer.crs:
        gdf = gdf.to_crs(base_layer.crs)
    if 'Shape_Length' in gdf.columns:
        gdf.drop('Shape_Length', axis=1, inplace=True)
    if 'Shape_Area' in gdf.columns:
        gdf.drop('Shape_Area', axis=1, inplace=True)

    # Perform spatial join (inner join by default)
    base_layer = gpd.sjoin(base_layer, gdf, how="left")
    
    # Drop the `index_right` column if it exists
    if 'index_right' in base_layer.columns:
        base_layer.drop('index_right', axis=1, inplace=True)

CPU times: total: 1min 10s
Wall time: 1min 10s


In [50]:
%%time
base_layer.to_file('h'+hexLevel+'_allAtOnce.gdb',driver='OpenFileGDB')

CPU times: total: 20.6 s
Wall time: 20.6 s


In [51]:
del layers_to_join, layer_gdfs, gdf, base_layer

# HEX 7

Hex imports

In [52]:
%%time
# Define the path to the geodatabase
inputs_gdb = r'C://Research/Grid_effort/H3Grid_Inputs.gdb'
hex_gdb = r"C://Research/Grid_effort/H3_5_10_Grd.gdb"
hexLevel = '7'

# Read in the H3_5 polygon grid layer as the base layer
base_layer = gpd.read_file(hex_gdb, layer='H3_'+hexLevel)
base_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)

CPU times: total: 28.5 s
Wall time: 28.5 s


Hex to counties

In [53]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='tj_2021_us_st_cnt')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

CPU times: total: 1.67 s
Wall time: 1.68 s


In [54]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

CPU times: total: 13.5 s
Wall time: 13.4 s


In [55]:
%%time
output_layer.to_file('h'+hexLevel+'_tj2021.gdb',driver='OpenFileGDB')

CPU times: total: 26 s
Wall time: 26.1 s


In [56]:
del joined_layer, output_layer

Hex to estuarine

In [57]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='Estuarine_Drainage_Areas')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

CPU times: total: 188 ms
Wall time: 179 ms


In [58]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

CPU times: total: 9.31 s
Wall time: 9.32 s


In [59]:
%%time
output_layer.to_file('h'+hexLevel+'_Estuarine.gdb',driver='OpenFileGDB')

CPU times: total: 22.7 s
Wall time: 22.7 s


In [60]:
del joined_layer, output_layer

Hex to huc8

In [61]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='WBDHU8')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

CPU times: total: 30 s
Wall time: 30 s


In [62]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

CPU times: total: 43.3 s
Wall time: 43.4 s


In [63]:
%%time
output_layer.to_file('h'+hexLevel+'_WBDHU8.gdb',driver='OpenFileGDB')

CPU times: total: 23.1 s
Wall time: 23.1 s


In [64]:
del joined_layer, output_layer

Hex to census

In [65]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='dtl_cnty_Census_ESRI')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

CPU times: total: 10.2 s
Wall time: 10.3 s


In [66]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

CPU times: total: 4min 12s
Wall time: 4min 12s


In [67]:
%%time
output_layer.to_file('h'+hexLevel+'_Census.gdb',driver='OpenFileGDB')

CPU times: total: 23.4 s
Wall time: 23.5 s


In [68]:
del joined_layer, output_layer

Hex to huc12

In [69]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='WBDHU12')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

CPU times: total: 9.11 s
Wall time: 9.09 s


In [70]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

CPU times: total: 9.39 s
Wall time: 9.38 s


In [71]:
%%time
output_layer.to_file('h'+hexLevel+'_WBDHU12.gdb',driver='OpenFileGDB')

CPU times: total: 27.9 s
Wall time: 27.9 s


In [72]:
del joined_layer, output_layer

Hex to all at once

In [73]:
%%time
layers_to_join = ['tj_2021_us_st_cnt', 'Estuarine_Drainage_Areas', 'WBDHU8', 'dtl_cnty_Census_ESRI', 'WBDHU12']
layer_gdfs = {layer: gpd.read_file(inputs_gdb, layer=layer) for layer in layers_to_join}

CPU times: total: 50.8 s
Wall time: 50.8 s


In [74]:
%%time
for name, gdf in layer_gdfs.items():
    # Ensure the CRS is the same between the base layer and the current layer
    if not gdf.crs == base_layer.crs:
        gdf = gdf.to_crs(base_layer.crs)
    if 'Shape_Length' in gdf.columns:
        gdf.drop('Shape_Length', axis=1, inplace=True)
    if 'Shape_Area' in gdf.columns:
        gdf.drop('Shape_Area', axis=1, inplace=True)

    # Perform spatial join (inner join by default)
    base_layer = gpd.sjoin(base_layer, gdf, how="left")
    
    # Drop the `index_right` column if it exists
    if 'index_right' in base_layer.columns:
        base_layer.drop('index_right', axis=1, inplace=True)

CPU times: total: 6min 18s
Wall time: 6min 18s


In [75]:
%%time
base_layer.to_file('h'+hexLevel+'_allAtOnce.gdb',driver='OpenFileGDB')

CPU times: total: 54.6 s
Wall time: 54.7 s


In [76]:
del layers_to_join, layer_gdfs, gdf, base_layer

# HEX 8

Hex imports

In [77]:
%%time
# Define the path to the geodatabase
inputs_gdb = r'C://Research/Grid_effort/H3Grid_Inputs.gdb'
hex_gdb = r"C://Research/Grid_effort/H3_5_10_Grd.gdb"
hexLevel = '8'

# Read in the H3_5 polygon grid layer as the base layer
base_layer = gpd.read_file(hex_gdb, layer='H3_'+hexLevel)
base_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)

CPU times: total: 3min 18s
Wall time: 3min 18s


Hex to counties

In [78]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='tj_2021_us_st_cnt')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

CPU times: total: 1.94 s
Wall time: 1.93 s


In [79]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

CPU times: total: 1min 38s
Wall time: 1min 39s


In [80]:
%%time
output_layer.to_file('h'+hexLevel+'_tj2021.gdb',driver='OpenFileGDB')

CPU times: total: 2min 59s
Wall time: 2min 59s


In [81]:
del joined_layer, output_layer

Hex to estuarine

In [82]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='Estuarine_Drainage_Areas')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

CPU times: total: 203 ms
Wall time: 196 ms


In [83]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

CPU times: total: 1min 29s
Wall time: 1min 29s


In [84]:
%%time
output_layer.to_file('h'+hexLevel+'_Estuarine.gdb',driver='OpenFileGDB')

CPU times: total: 2min 38s
Wall time: 2min 38s


In [85]:
del joined_layer, output_layer

Hex to huc8

In [86]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='WBDHU8')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

CPU times: total: 32.1 s
Wall time: 32.1 s


In [87]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

CPU times: total: 6min 2s
Wall time: 6min 2s


In [88]:
%%time
output_layer.to_file('h'+hexLevel+'_WBDHU8.gdb',driver='OpenFileGDB')

CPU times: total: 2min 35s
Wall time: 2min 35s


In [89]:
del joined_layer, output_layer

Hex to census

In [90]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='dtl_cnty_Census_ESRI')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

CPU times: total: 10.3 s
Wall time: 10.3 s


In [91]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

CPU times: total: 29min 51s
Wall time: 29min 54s


In [92]:
%%time
output_layer.to_file('h'+hexLevel+'_Census.gdb',driver='OpenFileGDB')

CPU times: total: 2min 39s
Wall time: 2min 39s


In [93]:
del joined_layer, output_layer

Hex to huc12

In [94]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='WBDHU12')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

CPU times: total: 9.31 s
Wall time: 9.34 s


In [95]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

CPU times: total: 57.9 s
Wall time: 58 s


In [96]:
%%time
output_layer.to_file('h'+hexLevel+'_WBDHU12.gdb',driver='OpenFileGDB')

CPU times: total: 2min 48s
Wall time: 2min 48s


In [97]:
del joined_layer, output_layer

Hex to all at once

In [98]:
%%time
layers_to_join = ['tj_2021_us_st_cnt', 'Estuarine_Drainage_Areas', 'WBDHU8', 'dtl_cnty_Census_ESRI', 'WBDHU12']
layer_gdfs = {layer: gpd.read_file(inputs_gdb, layer=layer) for layer in layers_to_join}

CPU times: total: 52.2 s
Wall time: 52.2 s


In [99]:
%%time
for name, gdf in layer_gdfs.items():
    # Ensure the CRS is the same between the base layer and the current layer
    if not gdf.crs == base_layer.crs:
        gdf = gdf.to_crs(base_layer.crs)
    if 'Shape_Length' in gdf.columns:
        gdf.drop('Shape_Length', axis=1, inplace=True)
    if 'Shape_Area' in gdf.columns:
        gdf.drop('Shape_Area', axis=1, inplace=True)

    # Perform spatial join (inner join by default)
    base_layer = gpd.sjoin(base_layer, gdf, how="left")
    
    # Drop the `index_right` column if it exists
    if 'index_right' in base_layer.columns:
        base_layer.drop('index_right', axis=1, inplace=True)

CPU times: total: 40min 26s
Wall time: 40min 28s


In [100]:
%%time
base_layer.to_file('h'+hexLevel+'_allAtOnce.gdb',driver='OpenFileGDB')

CPU times: total: 4min 21s
Wall time: 4min 21s


In [101]:
del layers_to_join, layer_gdfs, gdf, base_layer

# HEX 9

Hex imports

In [102]:
%%time
# Define the path to the geodatabase
inputs_gdb = r'C://Research/Grid_effort/H3Grid_Inputs.gdb'
hex_gdb = r"C://Research/Grid_effort/H3_5_10_Grd.gdb"
hexLevel = '9'

# Read in the H3_5 polygon grid layer as the base layer
base_layer = gpd.read_file(hex_gdb, layer='H3_'+hexLevel)
base_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)

CPU times: total: 23min 10s
Wall time: 23min 11s


Hex to counties

In [103]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='tj_2021_us_st_cnt')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

CPU times: total: 1.25 s
Wall time: 1.25 s


In [104]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

CPU times: total: 18min 5s
Wall time: 18min 5s


In [105]:
%%time
output_layer.to_file('h'+hexLevel+'_tj2021.gdb',driver='OpenFileGDB')

CPU times: total: 20min 13s
Wall time: 20min 14s


In [106]:
del joined_layer, output_layer

Hex to estuarine

In [107]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='Estuarine_Drainage_Areas')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

CPU times: total: 219 ms
Wall time: 230 ms


In [108]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

CPU times: total: 19min 8s
Wall time: 19min 9s


In [109]:
%%time
output_layer.to_file('h'+hexLevel+'_Estuarine.gdb',driver='OpenFileGDB')

CPU times: total: 18min 3s
Wall time: 18min 4s


In [110]:
del joined_layer, output_layer

Hex to huc8

In [111]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='WBDHU8')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

CPU times: total: 33.8 s
Wall time: 33.8 s


In [112]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

CPU times: total: 1h 6min 41s
Wall time: 1h 6min 44s


In [113]:
%%time
output_layer.to_file('h'+hexLevel+'_WBDHU8.gdb',driver='OpenFileGDB')

CPU times: total: 17min 53s
Wall time: 17min 54s


In [114]:
del joined_layer, output_layer

Hex to census

In [115]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='dtl_cnty_Census_ESRI')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

CPU times: total: 12.4 s
Wall time: 12.4 s


In [116]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

CPU times: total: 3h 55min 40s
Wall time: 3h 55min 53s


In [117]:
%%time
output_layer.to_file('h'+hexLevel+'_Census.gdb',driver='OpenFileGDB')

CPU times: total: 18min 7s
Wall time: 18min 8s


In [118]:
del joined_layer, output_layer

Hex to huc12

In [119]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='WBDHU12')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

CPU times: total: 7.83 s
Wall time: 7.85 s


In [120]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

CPU times: total: 6min 28s
Wall time: 6min 28s


In [121]:
%%time
output_layer.to_file('h'+hexLevel+'_WBDHU12.gdb',driver='OpenFileGDB')

CPU times: total: 19min 8s
Wall time: 19min 9s


In [122]:
del joined_layer, output_layer

Hex to all at once

In [123]:
%%time
layers_to_join = ['tj_2021_us_st_cnt', 'Estuarine_Drainage_Areas', 'WBDHU8', 'dtl_cnty_Census_ESRI', 'WBDHU12']
layer_gdfs = {layer: gpd.read_file(inputs_gdb, layer=layer) for layer in layers_to_join}

CPU times: total: 59.5 s
Wall time: 59.6 s


In [124]:
%%time
for name, gdf in layer_gdfs.items():
    # Ensure the CRS is the same between the base layer and the current layer
    if not gdf.crs == base_layer.crs:
        gdf = gdf.to_crs(base_layer.crs)
    if 'Shape_Length' in gdf.columns:
        gdf.drop('Shape_Length', axis=1, inplace=True)
    if 'Shape_Area' in gdf.columns:
        gdf.drop('Shape_Area', axis=1, inplace=True)

    # Perform spatial join (inner join by default)
    base_layer = gpd.sjoin(base_layer, gdf, how="left")
    
    # Drop the `index_right` column if it exists
    if 'index_right' in base_layer.columns:
        base_layer.drop('index_right', axis=1, inplace=True)

CPU times: total: 4h 34min 44s
Wall time: 4h 35min


In [125]:
%%time
base_layer.to_file('h'+hexLevel+'_allAtOnce.gdb',driver='OpenFileGDB')

CPU times: total: 26min 49s
Wall time: 26min 50s


In [126]:
del layers_to_join, layer_gdfs, gdf, base_layer

# HEX 10

Hex imports

In [127]:
%%time
# Define the path to the geodatabase
inputs_gdb = r'C://Research/Grid_effort/H3Grid_Inputs.gdb'
hex_gdb = r"C://Research/Grid_effort/H3_5_10_Grd.gdb"
hexLevel = '10'

# Read in the H3_5 polygon grid layer as the base layer
base_layer = gpd.read_file(hex_gdb, layer='H3_'+hexLevel)
base_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)

MemoryError: Unable to allocate 1.61 GiB for an array with shape (2, 108339425) and data type object

In [132]:
del hex_gdb

Hex to counties

In [128]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='tj_2021_us_st_cnt')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

CPU times: total: 2.86 s
Wall time: 3.09 s


In [129]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

NameError: name 'base_layer' is not defined

In [130]:
%%time
output_layer.to_file('h'+hexLevel+'_tj2021.gdb',driver='OpenFileGDB')

NameError: name 'output_layer' is not defined

In [131]:
del joined_layer, output_layer

NameError: name 'output_layer' is not defined

Hex to estuarine

In [ ]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='Estuarine_Drainage_Areas')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

In [ ]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

In [ ]:
%%time
output_layer.to_file('h'+hexLevel+'_Estuarine.gdb',driver='OpenFileGDB')

In [ ]:
del joined_layer, output_layer

Hex to huc8

In [ ]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='WBDHU8')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

In [ ]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

In [ ]:
%%time
output_layer.to_file('h'+hexLevel+'_WBDHU8.gdb',driver='OpenFileGDB')

In [ ]:
del joined_layer, output_layer

Hex to census

In [ ]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='dtl_cnty_Census_ESRI')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

In [ ]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

In [ ]:
%%time
output_layer.to_file('h'+hexLevel+'_Census.gdb',driver='OpenFileGDB')

In [ ]:
del joined_layer, output_layer

Hex to huc12

In [ ]:
%%time
joined_layer = gpd.read_file(inputs_gdb, layer='WBDHU12')
try:
    joined_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
except:
    print('Layer already cleared')

In [ ]:
%%time
if not joined_layer.crs == base_layer.crs:
        joined_layer = joined_layer.to_crs(base_layer.crs)

output_layer = gpd.sjoin(base_layer, joined_layer, how="left")

if 'index_right' in output_layer.columns:
        output_layer = output_layer.drop('index_right', axis=1)

In [ ]:
%%time
output_layer.to_file('h'+hexLevel+'_WBDHU12.gdb',driver='OpenFileGDB')

In [ ]:
del joined_layer, output_layer

Hex to all at once

In [ ]:
%%time
layers_to_join = ['tj_2021_us_st_cnt', 'Estuarine_Drainage_Areas', 'WBDHU8', 'dtl_cnty_Census_ESRI', 'WBDHU12']
layer_gdfs = {layer: gpd.read_file(inputs_gdb, layer=layer) for layer in layers_to_join}

In [ ]:
%%time
for name, gdf in layer_gdfs.items():
    # Ensure the CRS is the same between the base layer and the current layer
    if not gdf.crs == base_layer.crs:
        gdf = gdf.to_crs(base_layer.crs)
    if 'Shape_Length' in gdf.columns:
        gdf.drop('Shape_Length', axis=1, inplace=True)
    if 'Shape_Area' in gdf.columns:
        gdf.drop('Shape_Area', axis=1, inplace=True)

    # Perform spatial join (inner join by default)
    base_layer = gpd.sjoin(base_layer, gdf, how="left")
    
    # Drop the `index_right` column if it exists
    if 'index_right' in base_layer.columns:
        base_layer.drop('index_right', axis=1, inplace=True)

In [ ]:
%%time
base_layer.to_file('h'+hexLevel+'_allAtOnce.gdb',driver='OpenFileGDB')

In [ ]:
del layers_to_join, layer_gdfs, gdf, base_layer